In [1]:
import pandas as pd
from pathlib import Path

base_dir = Path.cwd()

customers = pd.read_csv(base_dir / "customers.csv")
orders = pd.read_csv(base_dir / "orders.csv")

customers.head(), orders.head()

(   customer_id     name         city
 0            1    Alice     New York
 1            2      Bob      Chicago
 2            3  Charlie  Los Angeles
 3            4    Diana      Seattle
 4            5     Eric       Boston,
    order_id  customer_id  amount  created_at
 0      1001            1     120  2024-01-01
 1      1002            1     300  2024-02-01
 2      1003            2     180  2024-02-15
 3      1004            3     500  2024-03-01
 4      1005            3     130  2024-03-10)

In [2]:
orders_with_customers = orders.merge(
    customers,
    on="customer_id",
    how="inner"
)

orders_with_customers.head()

,order_id,customer_id,amount,created_at,name,city
0,1001,1,120,2024-01-01,Alice,New York
1,1002,1,300,2024-02-01,Alice,New York
2,1003,2,180,2024-02-15,Bob,Chicago
3,1004,3,500,2024-03-01,Charlie,Los Angeles
4,1005,3,130,2024-03-10,Charlie,Los Angeles


In [3]:
customers_with_order_flag = customers.merge(
    orders[["customer_id"]],
    on="customer_id",
    how="left",
    indicator=True
)

customers_with_order_flag

,customer_id,name,city,_merge
0,1,Alice,New York,both
1,1,Alice,New York,both
2,2,Bob,Chicago,both
3,3,Charlie,Los Angeles,both
4,3,Charlie,Los Angeles,both
5,4,Diana,Seattle,both
6,4,Diana,Seattle,both
7,5,Eric,Boston,both


In [4]:
customer_spend = (
    orders_with_customers
    .groupby(["customer_id", "name"], as_index=False)
    .agg(total_spent=("amount", "sum"),
         order_count=("order_id", "count"))
    .sort_values("total_spent", ascending=False)
)

customer_spend

,customer_id,name,total_spent,order_count
2,3,Charlie,630,2
0,1,Alice,420,2
3,4,Diana,295,2
1,2,Bob,180,1
4,5,Eric,90,1


In [5]:
city_stats = (
    orders_with_customers
    .groupby("city", as_index=False)
    .agg(
        total_spent=("amount", "sum"),
        avg_order_value=("amount", "mean"),
        order_count=("order_id", "count")
    )
    .sort_values("total_spent", ascending=False)
)

city_stats

,city,total_spent,avg_order_value,order_count
2,Los Angeles,630,315.0,2
3,New York,420,210.0,2
4,Seattle,295,147.5,2
1,Chicago,180,180.0,1
0,Boston,90,90.0,1


In [6]:
orders_with_customers["created_at"] = pd.to_datetime(orders_with_customers["created_at"])
orders_with_customers["month"] = orders_with_customers["created_at"].dt.to_period("M").astype(str)

orders_with_customers[["order_id", "name", "amount", "month"]]

,order_id,name,amount,month
0,1001,Alice,120,2024-01
1,1002,Alice,300,2024-02
2,1003,Bob,180,2024-02
3,1004,Charlie,500,2024-03
4,1005,Charlie,130,2024-03
5,1006,Diana,75,2024-03
6,1007,Diana,220,2024-04
7,1008,Eric,90,2024-04


In [7]:
pivot = orders_with_customers.pivot_table(
    index="name",
    columns="month",
    values="amount",
    aggfunc="sum",
    fill_value=0
)

pivot

month,2024-01,2024-02,2024-03,2024-04
name,,,,
Alice,120,300,0,0
Bob,0,180,0,0
Charlie,0,0,630,0
Diana,0,0,75,220
Eric,0,0,0,90


In [8]:
output_path = base_dir / "customer_spend_summary.csv"
customer_spend.to_csv(output_path, index=False)
output_path

PosixPath('/Users/ethankeihan/code_repo/aws-data-engineering-30-days/02_pandas_polars/customer_spend_summary.csv')